In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM
from huggingface_hub import InferenceClient
from tqdm import tqdm
import torch
import json
import os
from torch.nn.utils.rnn import pad_sequence

In [3]:
data = []

with open('nsfw-tweets-en-ms.jsonl') as f:
    
    for x in tqdm(f):
        data.append(json.loads(x))

15715it [00:00, 319111.74it/s]


In [4]:
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.2')
model = AutoModelForCausalLM.from_pretrained(
    'mistralai/Mistral-7B-Instruct-v0.2', 
    use_flash_attention_2 = True, 
    torch_dtype = torch.float16,
    device_map="cuda"
)

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
def format_prompt(row):
    system_prompt = f"""

    text: {row['en']}

    If the text shows any sign of information, label it as 'informative'.
    If the text shows any sign of safe for work, label it as 'safe for work'.
    If the text shows any sign of lgbt, label it as 'lgbt'.
    If the text shows any sign of sexist, label it as 'sexist'.
    If the text shows any sign of violence, label it as 'violence'.
    If the text shows any sign of racist, label it as 'racist'.
    If the text shows any sign of porn, label it as 'porn'.
    If the text shows any sign of psychiatric or mental illness, label it as 'psychiatric or mental illness'.
    If the text shows any sign of self-harm, label it as 'self-harm'.
    If the text shows any sign of hate, label it as 'hate'.
    If the text shows any sign of religion insult, label it as 'religion insult'.
    If the text shows any sign of harassment, label it as 'harassment'.

    Only use the label from above choice.

    return the result in JSON format {'label', 'explain'}



    """

    messages = [{"role": "user", "content": f"{system_prompt}"}]

    encodeds = tokenizer.apply_chat_template(messages,tokenize = False)


    return encodeds

In [6]:
def format_prompt(row):
    system_prompt = f"""

    text: {row['Content']}

    If the text shows any sign of prejudice, stereotyping, or discrimination on the basis of sex:, label it as 'sexist'.
    If the text shows any sign of content that threatens, incites, glorifies, or expresses desire for violence or harm., label it as 'violence'.
    If the text shows any sign of prejudice, discrimination, or antagonism by an individual, community, or institution against a person or people on the basis of their membership of a particular racial or ethnic group, label it as 'racist'.
    If the text shows any sign of content depicting adult nudity or sexual behavior that is pornographic or intended to cause sexual arousal, label it as 'porn'.
    If the text shows any sign of psychiatric or mental illness, label it as 'psychiatric or mental illness'.
    If the text shows any sign of promotion, or otherwise encourage, suicide or self-harm, label it as 'self-harm'.
    If the text shows any sign of harassment, label it as 'harassment'.
    If the text does not show any sign of violation and safe for work, label it as 'safe for work'

    Only use the label from above choice.

    return the result in JSON format {'label', 'explain'}



    """

    messages = [{"role": "user", "content": f"{system_prompt}"}]

    encodeds = tokenizer.apply_chat_template(messages,tokenize = False)


    return encodeds

In [7]:
generate_kwargs = dict(
    temperature=1,
    max_new_tokens=4096,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("mesolitica/mallam-small-32768-fpf-v3",)

model = AutoModelForCausalLM.from_pretrained("mesolitica/mallam-small-32768-fpf-v3",
                                               use_flash_attention_2 = True, 
                                                torch_dtype = torch.float16).cuda()

tokenizer_config.json:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [12]:
%%time

batch_size = 5
tokenizer.pad_token = tokenizer.eos_token

with torch.no_grad():
    
    for i in tqdm(range(0, len(data), batch_size)):
        batch = data[i:i+batch_size]
        encodeds = [format_prompt(row) for row in batch]
        
        model_inputs = tokenizer(encodeds,padding=True,return_tensors="pt").to('cuda')

        generated_ids = model.generate(**model_inputs,temperature=1,max_new_tokens=1000,top_p=0.95,repetition_penalty=1.0,do_sample=True,pad_token_id = tokenizer.eos_token_id)
        decoded = tokenizer.batch_decode(generated_ids,skip_special_tokens=True,)
        
                with open('nsfw-inference-mistral.jsonl', 'a') as f:
            for i in range(len(decoded)):
                batch[i]['output'] = decoded[i].split('[/INST]')[1].strip()
                json.dump(batch[i], f)
                f.write('\n')

 10%|▉         | 301/3143 [23:11<3:39:02,  4.62s/it]

KeyboardInterrupt

